In [ ]:
# Install required libraries
!pip install flask flask-cors pyngrok nltk tensorflow

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Import libraries
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')

from nltk.stem import WordNetLemmatizer
import json
import pickle
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import SGD
import random
from flask import Flask, request, jsonify
from pyngrok import ngrok
from flask_cors import CORS

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Load intents file from Google Drive
file_path = '/content/drive/MyDrive/Semester 7/project_chatbot_2/intents.json'
data_file = open(file_path, 'r').read()
intents = json.loads(data_file)

# Tokenize and preprocess words
words = []
classes = []
documents = []
ignore_words = ['?', '!']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

# Prepare training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    bag = [1 if w in [lemmatizer.lemmatize(word.lower()) for word in doc[0]] else 0 for w in words]
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training, dtype=object)
train_x = list(training[:, 0])
train_y = list(training[:, 1])

# Build the model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.keras')  # Save model in .keras format

# Reload model and data for chatbot
model = load_model('chatbot_model.keras')  # Load the saved model

# Define chatbot logic
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words):
    sentence_words = clean_up_sentence(sentence)
    bag = [1 if w in sentence_words else 0 for w in words]
    return np.array(bag)

def predict_class(sentence, model):
    p = bow(sentence, words)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return [{"intent": classes[r[0]], "probability": str(r[1])} for r in results]

def getResponse(intents_list, intents_json):
    tag = intents_list[0]['intent']
    for i in intents_json['intents']:
        if i['tag'] == tag:
            return random.choice(i['responses'])

def chatbot_response(msg):
    ints = predict_class(msg, model)
    return getResponse(ints, intents)

# Create Flask app
app = Flask(__name__)
CORS(app)

@app.route("/chat", methods=["POST"])
def chat():
    data = request.get_json()
    message = data["message"]
    response = chatbot_response(message)
    return jsonify({"response": response})

# Expose app with ngrok
if __name__ == "__main__":
    ngrok.set_auth_token("2wOpwd2e5rlXXfTSyQpKiPF5s5D_5S67GioRMH2PAkkDNbkGs")  # Replace with your auth token
    public_url = ngrok.connect(5000, "http")
    print(f"Chatbot is running on: {public_url}/chat")
    app.run(port=5000)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Epoch 1/200
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0500 - loss: 3.3965
Epoch 2/200
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1421 - loss: 3.2710
Epoch 3/200
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2088 - loss: 3.1184
Epoch 4/200
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1101 - loss: 3.0955
Epoch 5/200
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2399 - loss: 2.8907
Epoch 6/200
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2421 - loss: 2.7357
Epoch 7/200
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3443 - loss: 2.5114
Epoch 8/200
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4134 - loss: 2.1561
Epoch 9/200
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4541 - loss: 2.1515
Epoch 10/200
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4920 - loss: 1.7448
Epoch 11/200
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4534 - loss: 1.8533
Epoch 12/200
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy:

INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [29/Apr/2025 13:38:33] "OPTIONS /chat HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step


...

INFO:werkzeug:127.0.0.1 - - [29/Apr/2025 13:52:43] "POST /chat HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step


INFO:werkzeug:127.0.0.1 - - [29/Apr/2025 13:52:47] "POST /chat HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Apr/2025 13:54:06] "OPTIONS /chat HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step


INFO:werkzeug:127.0.0.1 - - [29/Apr/2025 13:54:07] "POST /chat HTTP/1.1" 200 -
